In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from exp_analysis import *

In [3]:
case = 'light'

In [4]:
pars = {}
pars['heavy'] = {
    'm4' : [0.1, 0.2, 0.3, 0.4],
    'mz_prime': [1.25],
}
pars['light'] = {
    'm4' : [0.2, 0.3, 0.4], 
    'mz_prime': [0.03, 0.045, 0.06],
}

ctaus = [1, 5, 10, 20, 50, 100, 200]
!mkdir ../../fig/kde_test_2d/
save_folder = "../../fig/kde_test_2d/"
produce_samples = False

mkdir: cannot create directory ‘../../fig/kde_test_2d/’: File exists


In [5]:
#generation part
if produce_samples:
    for m4, mz_prime in itertools.product(pars[case]['m4'], pars[case]['mz_prime']):
        print(m4, mz_prime)
        if case == 'light':
            dark_gen_run = f'cd ..; ./dark_gen.py --M4 {m4} --mzprime {mz_prime} --UMU4 8e-9 --alpha_dark 0.25 --alpha_epsilon2 2e-10 --hierarchy light_mediator --neval 100000 --noplot'
        elif case == 'heavy':
            dark_gen_run = f'cd ..; ./dark_gen.py --M4 {m4} --mzprime {mz_prime} --UMU4 2.2e-7 --alpha_dark 0.4 --epsilon2 4.6e-4 --neval 100000 --noplot --hierarchy heavy_mediator'
        else:
            print('No good case selected')
        stream = os.popen(dark_gen_run)
        print(stream.read())

In [6]:
aux_gamma = []
aux_sigma = []
aux_n_evt = []
dfs = {}
for m4 in pars[case]['m4']:
    aux_gamma.append([])
    aux_sigma.append([])
    aux_n_evt.append([])
    for mz_prime in pars[case]['mz_prime']:
        dfs[m4] = pd.read_pickle(f'../data/nd280_nu/3plus1/m4_{m4}_mzprime_{mz_prime}/MC_m4_{m4}_mzprime_{mz_prime}.pckl')
        initialise_df(dfs[m4], ctaus=[25])

        aux_gamma[-1].append(dfs[m4]['weight_decay', ''][dfs[m4]['lead', '']].sum())
        aux_sigma[-1].append(dfs[m4]['adjusted_weight', ''][dfs[m4]['lead', '']].sum())
        aux_n_evt[-1].append(dfs[m4]['actual_weight', ''].sum())

aux_gamma = np.array(aux_gamma)
aux_sigma = np.array(aux_sigma)
aux_n_evt = np.array(aux_n_evt)

In [7]:
n_evt = 1000000
df_base = pd.read_pickle(f'/home/nic/Desktop/Dark_coherent/nicgen/data/nd280_nu/3plus1/scan/light_mediator/0.1_m4_0.5_0.01_mzprime_0.08_nevt_{n_evt}.pckl')
initialise_df(df_base, ctaus=[25], is_scan='m4_mz')

In [83]:
material = 'lead'
mask = df_base[material, ''] & (np.random.uniform(size=len(df_base)) > 0.986)
m4_values = df_base['m4', ''][mask].values
mz_values = df_base['mzprime', ''][mask].values
gamma_values = df_base['total_decay_rate', ''][mask].values
labels = {
    'x' : 'm_4 [GeV]',
    'y' : 'm_Z [GeV]',
    'z' : 'Decay rate [GeV]',
}
aux_mzprime, aux_m4 = np.meshgrid(pars[case]['mz_prime'], pars[case]['m4'])

In [84]:
import plotly.graph_objects as go
# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter3d(x=aux_m4.flatten(), y=aux_mzprime.flatten(), z=aux_gamma.flatten(), name="true", mode='markers')
)

fig.add_trace(
    go.Scatter3d(x=m4_values, y=mz_values, z=gamma_values, name="interpolation", mode='markers')
)

fig.update_layout(
    title=f'Decay rate on lead, N = {n_evt}',
    xaxis_title=labels['x'],
    yaxis_title=labels['y'],
    # zaxis_title=labels['z'],
)
fig.show()

In [8]:
# produce the gamma profile
material = 'lead'
mask = df_base[material, '']
m4_values = df_base['m4', ''][mask].values
mz_values = df_base['mzprime', ''][mask].values
weight_decay_values = df_base['weight_decay', ''][mask].values

m4_span = pars[case]['m4']
mz_span = np.linspace(mz_values.min(), mz_values.max(), 18)
grid_to_eval = np.stack(np.meshgrid(m4_span, mz_span, indexing='ij'), axis=-1)
df_values = np.stack([m4_values, mz_values], axis=-1)

smoothing_pars = [0.02, 0.005]
this_kde_weights = kde_Nd_weights(grid_to_eval, df_values, smoothing=smoothing_pars)
final_weights = this_kde_weights * weight_decay_values[:, np.newaxis, np.newaxis]
gamma_grid = np.sum(final_weights, axis=0)
gamma_std_grid = np.sqrt(np.sum(final_weights**2, axis=0))


for i, m4 in enumerate(pars[case]['m4']):
    plt.plot(pars[case]['mz_prime'], aux_gamma[i], 'o', label='true function')
    gamma_est = gamma_grid[i]
    gamma_std = gamma_std_grid[i]
    plt.plot(mz_span, gamma_est, label=f'kde 2d, $k_{{m_4}}$={smoothing_pars[0]:.2g}, $k_{{m_Z}}$={smoothing_pars[1]:.2g}')
    plt.fill_between(mz_span, gamma_est-gamma_std, gamma_est+gamma_std, alpha=0.25, color=plt.gca().lines[-1].get_color(), interpolate=True)

    bins = 20
    bin_width_m4 = 0.02
    this_mask = (m4_values > (m4-bin_width_m4/2)) & (m4_values < (m4+bin_width_m4/2))
    bin_range = [mz_values[this_mask].min(), mz_values[this_mask].max()]
    bin_span = (bin_range[1] - bin_range[0])
    bin_span_m4 = (m4_values.max() - m4_values.min())
    bin_width = bin_span/bins
    additional_weight = bin_span * bin_span_m4 / bin_width / bin_width_m4
    out = plt.hist(mz_values[this_mask], bins=bins, range=bin_range,
                weights=weight_decay_values[this_mask]*additional_weight,
                histtype='step', label='histogram')

    plt.legend(frameon=False, loc='upper right')
    plt.xlabel(r"$M_Z'$ [GeV]")
    plt.ylabel('Decay rate [GeV]')
    # plt.yscale('log')
    # plt.ylim(0, 1e-7)
    # plt.xlim(bin_range)
    plt.title(f'N random = {n_evt}\n{case} mediator, $m_4$ = {m4} GeV, only {material}')
    plt.tight_layout()
    plt.savefig(save_folder+f'gamma_{material}_nevt_{n_evt}_m4_{m4}.png', dpi=250)
    plt.close()

In [29]:
# produce the sigma profile
adjusted_weight = df_base['adjusted_weight', ''][mask].values
final_weights = this_kde_weights * adjusted_weight[:, np.newaxis, np.newaxis]
sigma_grid = np.sum(final_weights, axis=0)
sigma_std_grid = np.sqrt(np.sum(final_weights**2, axis=0))

for i, m4 in enumerate(pars[case]['m4']):
    plt.plot(pars[case]['mz_prime'], aux_sigma[i], 'o', label='true function')
    sigma_est = sigma_grid[i]
    sigma_std = sigma_std_grid[i]
    plt.plot(mz_span, sigma_est, label=f'kde 2d, $k_{{m_4}}$={smoothing_pars[0]:.2g}, $k_{{m_Z}}$={smoothing_pars[1]:.2g}')
    plt.fill_between(mz_span, sigma_est-sigma_std, sigma_est+sigma_std, alpha=0.25, color=plt.gca().lines[-1].get_color(), interpolate=True)

    bins = 20
    bin_width_m4 = 0.02
    this_mask = (m4_values > (m4-bin_width_m4/2)) & (m4_values < (m4+bin_width_m4/2))
    bin_range = [mz_values[this_mask].min(), mz_values[this_mask].max()]
    bin_span = (bin_range[1] - bin_range[0])
    bin_span_m4 = (m4_values.max() - m4_values.min())
    bin_width = bin_span/bins
    additional_weight = bin_span * bin_span_m4 / bin_width / bin_width_m4
    print(mz_values[this_mask], additional_weight, adjusted_weight[this_mask])
    print(bins, bin_range)
    out = plt.hist(mz_values[this_mask], bins=bins, range=bin_range,
                weights=adjusted_weight[this_mask]*additional_weight,
                histtype='step', label='histogram')

    plt.legend(frameon=False, loc='upper right')
    plt.xlabel(r"$M_Z'$ [GeV]")
    plt.ylabel('Cross section [GeV$^{-2}$]')
    # plt.yscale('log')
    plt.ylim(0, 1.2*max(sigma_est))
    # plt.xlim(bin_range)
    plt.title(f'N random = {n_evt}\n{case} mediator, $m_4$ = {m4} GeV, only {material}')
    plt.tight_layout()
    plt.savefig(save_folder+f'sigma_{material}_nevt_{n_evt}_m4_{m4}.png', dpi=250)
    plt.close()

[0.01021936 0.01023621 0.01001331 ... 0.03004658 0.0330881  0.03143405] 399.9993627363359 [2.58924625e-52 2.25412240e-52 2.38374660e-52 ... 8.63817388e-54
 1.45909753e-53 8.87275858e-53]
20 [0.010000221272559173, 0.07983607358588231]
[0.0100715  0.01026426 0.01027422 ... 0.02994336 0.03208134 0.05782309] 399.99936273633585 [1.61274518e-53 1.91502087e-53 1.39061462e-53 ... 2.50056209e-53
 3.19949172e-53 1.16487076e-52]
20 [0.010000042206786719, 0.07987399672128147]
[0.01040714 0.01007085 0.01035518 ... 0.02970324 0.03040311 0.04034072] 399.99936273633585 [8.69838986e-56 2.66443164e-57 1.25308185e-54 ... 2.18836659e-58
 6.31872832e-54 1.44313619e-53]
20 [0.010000190905065693, 0.07993479853877426]


In [31]:
# produce the n_evt profile at different stages in the selection
actual_weight = df_base['actual_weight', ''][mask].values
final_weights = this_kde_weights * actual_weight[:, np.newaxis, np.newaxis]
n_event_grid = np.sum(final_weights, axis=0)
n_event_std_grid = np.sqrt(np.sum(final_weights**2, axis=0))

ctau = 25
for mask in ['trivial', 'selected', f'decay_in_tpc_{ctau}', f'total_selected_{ctau}']:
    for i, m4 in enumerate(pars[case]['m4']):
        plt.plot(pars[case]['mz_prime'], aux_n_evt[i], 'o', label='true function')
        n_event_est = n_event_grid[i]
        n_event_std = n_event_std_grid[i]
        plt.plot(mz_span, n_event_est, label=f'kde 2d, $k_{{m_4}}$={smoothing_pars[0]:.2g}, $k_{{m_Z}}$={smoothing_pars[1]:.2g}')
        plt.fill_between(mz_span, n_event_est-n_event_std, n_event_est+n_event_std, alpha=0.25, color=plt.gca().lines[-1].get_color(), interpolate=True)

        bins = 20
        bin_width_m4 = 0.02
        this_mask = (m4_values > (m4-bin_width_m4/2)) & (m4_values < (m4+bin_width_m4/2))
        bin_range = [mz_values[this_mask].min(), mz_values[this_mask].max()]
        bin_span = (bin_range[1] - bin_range[0])
        bin_span_m4 = (m4_values.max() - m4_values.min())
        bin_width = bin_span/bins
        additional_weight = bin_span * bin_span_m4 / bin_width / bin_width_m4
        out = plt.hist(mz_values[this_mask], bins=bins, range=bin_range,
                    weights=actual_weight[this_mask]*additional_weight,
                    histtype='step', label='histogram')

        plt.legend(frameon=False, loc='upper right')
        plt.xlabel(r"$M_Z'$ [GeV]")
        plt.ylabel('Expected number of entries')
        # plt.yscale('log')
        # plt.ylim(0, 1e-7)
        # plt.xlim(bin_range)
        plt.title(f'N random = {n_evt}\n{case} mediator, $m_4$ = {m4} GeV, only {material}')
        plt.tight_layout()
        plt.savefig(save_folder+f'mask_{mask}_nevt_{n_evt}_m4_{m4}.png', dpi=250)
        plt.close()

[0.01021936 0.01023621 0.01001331 ... 0.03004658 0.0330881  0.03143405] 399.9993627363359 [2.58924625e-52 2.25412240e-52 2.38374660e-52 ... 8.63817388e-54
 1.45909753e-53 8.87275858e-53]
20 [0.010000221272559173, 0.07983607358588231]
[0.0100715  0.01026426 0.01027422 ... 0.02994336 0.03208134 0.05782309] 399.99936273633585 [1.61274518e-53 1.91502087e-53 1.39061462e-53 ... 2.50056209e-53
 3.19949172e-53 1.16487076e-52]
20 [0.010000042206786719, 0.07987399672128147]
[0.01040714 0.01007085 0.01035518 ... 0.02970324 0.03040311 0.04034072] 399.99936273633585 [8.69838986e-56 2.66443164e-57 1.25308185e-54 ... 2.18836659e-58
 6.31872832e-54 1.44313619e-53]
20 [0.010000190905065693, 0.07993479853877426]
[0.01021936 0.01023621 0.01001331 ... 0.03004658 0.0330881  0.03143405] 399.9993627363359 [2.58924625e-52 2.25412240e-52 2.38374660e-52 ... 8.63817388e-54
 1.45909753e-53 8.87275858e-53]
20 [0.010000221272559173, 0.07983607358588231]
[0.0100715  0.01026426 0.01027422 ... 0.02994336 0.03208134 0.